In [1]:
## data reading

from ipynb.fs.full.my_functions import *
## importa biblioteca do Kane para ler os emails

from pandas import DataFrame
from pandas import concat
import numpy as np
from matplotlib import *


spam = DataFrame({'message': [], 'class': []})
ham = DataFrame({'message': [], 'class': []})

spam = dataFrameFromDirectory('../../emails/spam', 'spam')
ham = dataFrameFromDirectory('../../emails/ham', 'ham')

spam.head()
ham.head()

,message,class
../../emails/ham/01843.ce0f26c22efed433edbce217b1e56331,URL: http://scriptingnews.userland.com/backiss...,ham
../../emails/ham/02297.a0f08a4cc518d9c666dffef38f4b3076,"URL: http://www.newsisfree.com/click/-6,858259...",ham
../../emails/ham/00062.009f5a1a8fa88f0b38299ad01562bb37,(This list is sponsored by Ironclad Networks h...,ham
../../emails/ham/02202.fd6cdebf0762194039aed87bb12a8cc9,URL: http://boingboing.net/#85518715\n\nDate: ...,ham
../../emails/ham/00555.4139b207075574b3774942a3a42013d8,">>>>> ""U"" == Udhay Shankar N <Udhay> writes:\n...",ham


In [2]:
print(ham.shape[0])
print(spam.shape[0])


2500
500


In [3]:
## data preparation

## randomizando spam e ham para eliminar bias

from sklearn.utils import shuffle

ham  = shuffle(ham)
spam = shuffle(spam)

#spliting into train (80%) and test (20%)

train = ham[:int(ham.shape[0]*.8)]
train = train.append(spam[:int(spam.shape[0]*.8)])

test = ham[int(ham.shape[0]*.8):]
test = test.append(spam[int(spam.shape[0]*.8):])

## desnecessario, soh pra dar mais uma embaralhada
train = shuffle(train)
test  = shuffle(test)

print(train.shape)
print(test.shape)

print(train.head())
print(test.head())

print("treino:\n",train["class"].value_counts())
print("teste:\n",test["class"].value_counts())

(2400, 2)
(600, 2)
                                                                                              message  \
../../emails/ham/02020.c4892988e3528bb4cd4fa0e1...  URL: http://www.newsisfree.com/click/-4,828887...   
../../emails/ham/01738.780eecf7f0db1db00bd0a624...  [Skip Montanaro]\n\n> Because I get mail throu...   
../../emails/spam/00154.b6c448ccff434e2dbe2c7c2...  On January 1st 2002, the European countries be...   
../../emails/ham/01813.2711af4b8a31e4ff0c38fe92...  \n\nForwarded-by: Monty Solomon <monty@roscom....   
../../emails/ham/00606.9733a34d34069bbc9671b029...  \n\n\n\n> All else being equal, the terminal v...   

                                                   class  
../../emails/ham/02020.c4892988e3528bb4cd4fa0e1...   ham  
../../emails/ham/01738.780eecf7f0db1db00bd0a624...   ham  
../../emails/spam/00154.b6c448ccff434e2dbe2c7c2...  spam  
../../emails/ham/01813.2711af4b8a31e4ff0c38fe92...   ham  
../../emails/ham/00606.9733a34d34069bbc9671b029...   ha

In [9]:
## data preparation #2: mapping message into vector

from sklearn.feature_extraction.text import CountVectorizer

## vetorizando a base (mudanca da representação do email = vetor no espaço de palavras)
## o classificador vai rodar sobre esse objeto/vetor versus valores "class"

vectorizer = CountVectorizer()

#print("Vocabulary: ", vectorizer.vocabulary_) 

trainvect = vectorizer.fit_transform(train['message'].values)
traintarg = train['class'].values

testvect = vectorizer.transform(test['message'].values)
testtarg = test['class'].values

##check de consistencia

print("\ntreino:\n",trainvect.shape,"\n ham/spam:",sum(traintarg=="ham"),"/",sum(traintarg=="spam"))
print("\nteste:\n",testvect.shape,"\n ham/spam:",sum(testtarg=="ham"),"/",sum(testtarg=="spam"))


treino:
 (2400, 58500) 
 ham/spam: 2000 / 400

teste:
 (600, 58500) 
 ham/spam: 500 / 100


In [5]:
## model training sobre o subconjunto de treino

from sklearn.naive_bayes import MultinomialNB

## classifier é o objeto [Multinomial Naive Bayes] que armazena o modelo

classifier = MultinomialNB()
classifier.fit(trainvect, traintarg)

print(trainvect.shape)
print(traintarg)

(2400, 58500)
['ham' 'ham' 'spam' ... 'ham' 'ham' 'spam']


In [13]:
## rodando o modelo treinado sobre o subconjunto de teste

predictions = classifier.predict(testvect)
#print(predictions)
#print(predictions.shape)

right = sum(testtarg == predictions)
wrong =  predictions.shape[0] - right

print('acertos: ', right , "/", predictions.shape[0],
      '= ', "{:.1f}".format(right/predictions.shape[0]*100),"%")

## classificando erros 
i = 0
fnct = 0
fpct = 0
for each in predictions:
    ## falso negativo
    if each == 'ham' and testtarg[i] == 'spam':
        fnct += 1

    ## falso positivo
    else: 
        if each == 'spam' and testtarg[i] == 'ham':
            fpct += 1
    i+=1

totfs = fpct + fnct 
#print ("check:",totfs,wrong )
#print ('falso negativo =', fnct,"= ","{:.1f}".format(100*fnct/wrong),"%",
#       '\nfalso positivo = ',fpct,"=","{:.1f}".format(100*fpct/wrong),"%")

print ('falso negativo =', fnct,"= ","{:.1f}".format(100*fnct/predictions.shape[0]),"%",
       '\nfalso positivo = ',fpct,"=","{:.1f}".format(100*fpct/predictions.shape[0]),"%")

acertos:  565 / 600 =  94.2 %
falso negativo = 33 =  5.5 % 
falso positivo =  2 = 0.3 %


In [14]:
## redundancia: usando calculo via sklearn

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

acc_score = accuracy_score(testtarg,predictions)

print("\ncoeficiente de acurácia:")
print(acc_score)

conf_mat = confusion_matrix(testtarg, predictions)

print("\nmatriz de confusão:")
print(conf_mat)



coeficiente de acurácia:
0.9416666666666667

matriz de confusão:
[[498   2]
 [ 33  67]]


In [15]:
## curiosidades
## dummy test

testes = ['xxxx91293712631027216302131','Free viagra', "sex for free","pay for sex", "I love you","enlarge your penis"]
example_counts = vectorizer.transform(testes)
print(example_counts.shape)
predictions = classifier.predict(example_counts)
print(predictions)

(6, 58500)
['ham' 'spam' 'ham' 'ham' 'ham' 'spam']
